# **Tugas Pertemuan 1 - Crawling berita online**

Crawling berita online detik.com

Crawling data atau web crawling adalah proses ekstraksi informasi dari situs web dengan menggunakan program komputer. Tujuan utama dari web crawling adalah untuk mengumpulkan data dari berbagai sumber di internet.
Terdapat berbagai macam library yang dapat digunakan untuk crawling data. Library yang digunakan untuk crawling berita detik.com yaitu Beautiful Soup dan Requests. Beautiful Soup Digunakan untuk mengekstrak informasi dari HTML dan Request digunakan untuk mengirim permintaan HTTP dan mengakses halaman web.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"


Selanjutnya kumpulkan data berita dari website Detik.com dan simpan dalam format CSV.

- Proses pengambilan data dengan fungsi get_data  digunakan untuk mengambil artikel  dari halaman yang ditentukan. Kode ini bertujuan crawl URL, mem-parsing HTML untuk menemukan artikel, dan kemudian mengambil detail artikel (judul, tanggal, isi berita, kategori) dari setiap tautan artikel yang ditemukan.

- Pembersihan Konten/data menggunakan fungsi clean_content untuk menghapus elemen yang tidak di inginkan.

- Save Data artikel (judul, tanggal, isi berita, kategori) disimpan dalam daftar list dan kemudian dikumpulkan dalam DataFrame Pandas.

In [12]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Fungsi untuk membersihkan konten berita
def clean_content(content_element):
    if not content_element:
        return "Content Not Found"
    paragraphs = content_element.find_all("p")
    return " ".join(p.text.strip() for p in paragraphs)

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, category, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        # Cek apakah jumlah artikel untuk kategori sudah mencukupi
        if len([k for k in kategori_list if k == category]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = clean_content(content_element)

        # Simpan data ke dalam list global
        judul.append(title)
        tanggal.append(date)
        isi_berita.append(content)
        kategori_list.append(category)

        # Debug: Tampilkan judul artikel untuk verifikasi
        if len(judul) <= 10:
            print(title)

        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list URL dan kategori yang akan di-crawl
base_urls = [
    "https://sport.detik.com/indeks",
    "https://food.detik.com/indeks",
]
categories = [
    "Olahraga",
    "Makanan",
]

# Inisialisasi list untuk menyimpan data
judul = []
isi_berita = []
tanggal = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap URL dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat DataFrame dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi_berita": isi_berita,
    "kategori": kategori_list,
})

# Menyimpan DataFrame ke file CSV
df.to_csv("data_berita_detik.csv", index=False)

# Menampilkan DataFrame
print(df)


Jorge Martin Bisa Saingi Bagnaia-Marc Marquez di 2025?
Tiwi Tak Ingin Sia-Siakan Kesempatan Main di World Tour Finals
Daftar Atlet BWF World Tour Finals 2024, 6 Wakil dari RI
Performanya Terus Menanjak, Putri KW Bungkam Kritik
Marc Marquez Sukses Usir Hantu
Tersandung Pemerkosaan, Karier Conor McGregor di UFC Tamat?
Gresini, Tempat Marc Marquez Reborn
PBSI Bersyukur Atlet Non-Pelatnas Lolos ke World Tour Finals 2024
Indonesia Punya 6 Wakil di BWF World Tour Finals 2024
Mengapresiasi Brad Binder, Rider Paling Top Tak Pakai Ducati
                                                judul  \
0   Jorge Martin Bisa Saingi Bagnaia-Marc Marquez ...   
1   Tiwi Tak Ingin Sia-Siakan Kesempatan Main di W...   
2   Daftar Atlet BWF World Tour Finals 2024, 6 Wak...   
3   Performanya Terus Menanjak, Putri KW Bungkam K...   
4                      Marc Marquez Sukses Usir Hantu   
..                                                ...   
95  5 Tips Menyeduh Kopi di Musim Hujan agar Rasan...   
96  Ini 1

Data yang diambil terdiri dari :
- Judul
- Isi berita
- Tanggal
- Kategori

Pada tahap ini menampilkan data tersebut dan diambil berdasarkan kategori
***Kesehatan*** dan ***Kuliner*** dan hanya mengambil 100 sampel data crawl

Berikut untuk 5 sampel data berdasarkan kategori ***Olahraga***

In [3]:
df=pd.read_csv("data_berita_detik.csv")
df.head(5)

,judul,isi_berita,tanggal,kategori
0,Jorge Martin Bisa Saingi Bagnaia-Marc Marquez ...,"Madrid - Jorge Martin mengatakan, Francesc...","Kamis, 28 Nov 2024 17:50 WIB",Olahraga
1,Tiwi Tak Ingin Sia-Siakan Kesempatan Main di W...,Jakarta - Pebulutangkis ganda putri Amallia...,"Kamis, 28 Nov 2024 12:30 WIB",Olahraga
2,"Daftar Atlet BWF World Tour Finals 2024, 6 Wak...",Daftar Isi \n \n \n \n \r\n\t\t\t\t\t\t\tPemai...,"Rabu, 27 Nov 2024 18:50 WIB",Olahraga
3,"Performanya Terus Menanjak, Putri KW Bungkam K...",Jakarta - Putri Kusuma Wardani sempat dapat...,"Rabu, 27 Nov 2024 17:20 WIB",Olahraga
4,Marc Marquez Sukses Usir Hantu,"Jakarta - Semusim di tim Gresini , Marc Ma...","Rabu, 27 Nov 2024 16:15 WIB",Olahraga


Berikut untuk 5 sampel data berdasarkan kategori ***Makanan***

In [4]:
df.tail(5)

,judul,isi_berita,tanggal,kategori
95,Ini 10 Restoran Klasik Langganan 'Old Money' d...,Jakarta - Sudah berdiri sejak puluhan tahun l...,"Rabu, 27 Nov 2024 06:00 WIB",Makanan
96,"Disebut Menu Diet, 6 Makanan dan Minuman Ini M...",Jakarta -\n \n Beberapa produk...,"Rabu, 27 Nov 2024 05:00 WIB",Makanan
97,Jago Masak! Ini Momen Mahalini Masak bareng Ri...,"Jakarta - Menjadi seorang istri, Mahalini ter...","Selasa, 26 Nov 2024 22:00 WIB",Makanan
98,Keseruan Lucky Hakim Saat Cicip Durian hingga ...,"Jakarta - Setelah sempat mundur, Lucky Hakim ...","Selasa, 26 Nov 2024 21:00 WIB",Makanan
99,Ini 5 Aksi Chef yang Bikin Heboh dan Picu Kont...,Jakarta -\n \n Bekerja sebagai...,"Selasa, 26 Nov 2024 19:30 WIB",Makanan
